In [1]:
import re
from collections import defaultdict

def parse_input(input_lines):
    filesystem = defaultdict(lambda: {"type": None, "parent": None, "children": {}, "size": 0})
    cmd_regex = re.compile(r'\$(?:\s(cd|ls))?(?:\s(.*))?')
    ls_response_regex = re.compile(r'(\d+|\bdir\b)\s+(\w+)')
    current_dir = "/"
    filesystem[current_dir]["type"] = "dir"

    for line in input_lines:
        cmd_match = cmd_regex.match(line)
        if cmd_match:
            cmd, arg = cmd_match.groups()
            if cmd == "cd":
                if arg == "..":
                    current_dir = filesystem[current_dir]["parent"]
                elif arg == "/":
                    current_dir = "/"
                else:
                    current_dir += "/" + arg
        else:
            file_type, name = ls_response_regex.match(line).groups()
            path = current_dir + "/" + name
            if file_type == "dir":
                filesystem[path]["type"] = "dir"
                filesystem[path]["parent"] = current_dir
                filesystem[current_dir]["children"][path] = filesystem[path]
            else:
                filesystem[path]["type"] = "file"
                filesystem[path]["size"] = int(file_type)
                filesystem[current_dir]["children"][path] = filesystem[path]
    return filesystem

def calculate_sizes(filesystem, path):
    if filesystem[path]["type"] == "file":
        return filesystem[path]["size"]
    total_size = 0
    for child_path, child in filesystem[path]["children"].items():
        child_size = calculate_sizes(filesystem, child_path)
        total_size += child_size
    filesystem[path]["size"] = total_size
    return total_size

def solve(input_lines):
    filesystem = parse_input(input_lines)
    calculate_sizes(filesystem, "/")
    small_dirs = [dir for dir in filesystem.values() if dir["type"] == "dir" and dir["size"] <= 100000]
    total_size = sum(dir["size"] for dir in small_dirs)
    return total_size

def main():
    with open('Day7.txt', 'r') as file:
        input_lines = file.read().splitlines()
    print(solve(input_lines))

if __name__ == "__main__":
    main()


1555642


In [2]:
def solve_part_two(input_lines):
    filesystem = parse_input(input_lines)
    calculate_sizes(filesystem, "/")
    
    total_disk_space = 70000000
    used_space = filesystem["/"]["size"]
    unused_space = total_disk_space - used_space
    required_space = 30000000

    # The amount of additional space required to reach the required_space
    additional_required_space = required_space - unused_space

    # Only consider directories big enough to meet the additional space requirements
    big_enough_dirs = [dir for dir in filesystem.values() if dir["type"] == "dir" and dir["size"] >= additional_required_space]

    # Find the smallest among the big enough directories
    smallest_dir = min(big_enough_dirs, key=lambda dir: dir["size"])

    return smallest_dir["size"]

def main():
    with open('Day7.txt', 'r') as file:
        input_lines = file.read().splitlines()
    print(solve_part_two(input_lines))

if __name__ == "__main__":
    main()


5974547
